- This scraper takes the username of a streamer and based on that scraps the url 
https://twitchtracker.com/games


In [241]:
#imports 
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sys

In [244]:
sys.setrecursionlimit(10**6)
def parse_games_2(user_name):
    
    """ This function scrapes the game of a streamer as a input in the source Twitchtracker"""

    user_name.lower()
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url = f"https://twitchtracker.com/{user_name}/games", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    
    df = df.rename(columns={"Game": "game", "Avg. viewers": "avgViewers", "Max. viewers": "maxViewers", "Followers": "followersPerHour", "Total airtime": "totalAirtime", "Last seen": "lastSeen"})
    df["totalAirtime"] = df["totalAirtime"].astype(str).str[:-1]
    df["followersPerHour"] = df.followersPerHour.str.rstrip("/hour")
    df[["absoluteTotalAirtime", "percentageOfTotalAirtime", "none"]] = df.totalAirtime.str.split(" ", expand=True)

    
    df["percentageOfTotalAirtime"] = df.percentageOfTotalAirtime.str.strip()
    df["percentageOfTotalAirtime"] = df["percentageOfTotalAirtime"].replace(to_replace="", value=np.nan)
    
    #df.drop(labels="none", axis=1, inplace=True)
    df.drop(labels="totalAirtime", axis=1, inplace=True)

    df["absoluteTotalAirtime"] = df["absoluteTotalAirtime"].astype('int32')
    
    df["percentageOfTotalAirtime"] = df["percentageOfTotalAirtime"].astype('float64')
    #df["percentageOfTotalAirtime"] = df["percentageOfTotalAirtime"].apply(lambda x: float(x.split().replace(',', '')))
    
    df["followersPerHour"] = df["followersPerHour"].astype('float64')
    df["percentageOfTotalAirtime"] = df["percentageOfTotalAirtime"].where(df["percentageOfTotalAirtime"] > 0)
    df.dropna(subset = ["percentageOfTotalAirtime"], axis=0, how="any", inplace=True)
    
    df["lastSeen"] = pd.to_datetime(df["lastSeen"])
    return df



In [245]:
df = parse_games_2("Shroud")

In [246]:
df

,#,game,avgViewers,maxViewers,followersPerHour,lastSeen,absoluteTotalAirtime,percentageOfTotalAirtime,none
0,#1,PLAYERUNKNOWN'S BATTLEGROUNDS,29080,152919,665.0,2021-04-02 06:00:00,157330,27.3,None
1,#2,Counter-Strike: Global Offensive,13602,104421,309.4,2021-04-09 23:40:00,40344,7.0,None
2,#3,Apex Legends,48327,202955,1185.3,2021-05-05 20:00:00,36546,6.3,None
3,#4,VALORANT,54721,516289,1558.4,2021-06-01 21:50:00,32646,5.7,None
4,#5,World of Warcraft,19266,51344,279.1,2021-06-03 09:00:00,31188,5.4,None
...,...,...,...,...,...,...,...,...,...
101,#102,Sniper Ghost Warrior Contracts 2,25034,29248,246.8,2021-05-22 04:10:00,310,0.1,None
102,#103,World War Z,35361,41819,417.2,2019-04-20 02:40:00,300,0.1,None
103,#104,Tom Clancy's Ghost Recon: Wildlands,20672,27493,254.3,2019-05-09 18:30:00,298,0.1,None
104,#105,God of War (1),22562,30554,277.6,2020-11-18 03:20:00,295,0.1,None


In [240]:
df.dtypes

#                                   object
game                                object
avgViewers                           int64
maxViewers                           int64
followersPerHour                   float64
lastSeen                    datetime64[ns]
absoluteTotalAirtime                 int32
percentageOfTotalAirtime           float64
none                                object
dtype: object

In [202]:
df.dtypes

#                            object
game                         object
avgViewers                    int64
maxViewers                    int64
followers                   float64
lastSeen                     object
absoluteTotalAirtime          int32
percentageOfTotalAirtime    float64
dtype: object

'asdff dd'

In [ ]:
new_string